In [1]:
import numpy as np
import pandas as pd
from pandas.core.indexes.datetimes import DatetimeIndex
from pandas.core.frame import DataFrame
from pandas.core.groupby.generic import DataFrameGroupBy

from pathlib import Path, PosixPath

In [2]:
class HandleData:
    def __init__(self) -> None:
        self.data_path = Path('../data/')
        self.day_name = self.__get_day_name()
        self.min_str = self.__get_min_str()
    
    def __get_day_name(self) -> DatetimeIndex:
        return pd.date_range(
            start='2020-09-01',
            end='2020-11-30',
            freq='D'
        ).drop(
            pd.date_range(
                start='2020-10-01',
                end='2020-10-08',
                freq='D'
            )
        )
    
    def __get_min_str(self) -> DatetimeIndex:
        return ['{}{}'.format(
            t.split(' ')[1].split(':')[0],
            t.split(' ')[1].split(':')[1]
        ) for t in pd.date_range(
            start='00:00',
            end='23:55',
            freq='5Min'
        ).astype(str)]
    
    def __pretreatment(self, file: PosixPath) -> DataFrame:
        df = pd.read_excel(Path.joinpath(self.data_path, file.name))
        df = df.replace(0, np.nan)\
                .drop(['Unnamed: 0'], axis=1)\
                .drop(['time'], axis=1)\
                .drop(['1001'], axis=1).drop(['1002'], axis=1).drop(['1003'], axis=1).drop(['1004'], axis=1)\
                .drop(['1005'], axis=1).drop(['1006'], axis=1).drop(['1007'], axis=1).drop(['1008'], axis=1)
        
        df.columns = self.day_name
        df.columns = df.columns.day_name()
        
        return df
    
    def __grouping(self, groups: DataFrameGroupBy) -> DataFrame:
        mon_mean = pd.DataFrame(groups.get_group('Monday').mean())
        mon_mean.index = self.min_str
        mon_mean.columns = ['Monday']

        tue_mean = pd.DataFrame(groups.get_group('Tuesday').mean())
        tue_mean.index = self.min_str
        tue_mean.columns = ['Tuesday']

        wed_mean = pd.DataFrame(groups.get_group('Wednesday').mean())
        wed_mean.index = self.min_str
        wed_mean.columns = ['Wednesday']

        thur_mean = pd.DataFrame(groups.get_group('Thursday').mean())
        thur_mean.index = self.min_str
        thur_mean.columns = ['Thursday']

        fri_mean = pd.DataFrame(groups.get_group('Friday').mean())
        fri_mean.index = self.min_str
        fri_mean.columns = ['Friday']

        sat_mean = pd.DataFrame(groups.get_group('Saturday').mean())
        sat_mean.index = self.min_str
        sat_mean.columns = ['Saturday']

        sun_mean = pd.DataFrame(groups.get_group('Sunday').mean())
        sun_mean.index = self.min_str
        sun_mean.columns = ['Sunday']
        
        return pd.concat([mon_mean, tue_mean, wed_mean, thur_mean, fri_mean, sat_mean, sun_mean], axis=1)
        
    
    def run(self) -> None:
        for file in self.data_path.iterdir():
            if file.suffix == '.xls':
                df = self.__pretreatment(file).T
                
                groups = df.groupby(df.index)
                result = self.__grouping(groups)
                result.to_excel(
                    './星期/{}x'.format(file.name)          

In [3]:
def main() -> None:
    HandleData().run()

In [4]:
if __name__ == '__main__':
    main()